In [5]:
# Load required packagesImpor

# Jupyter notebook settings
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # Show all outputs, not only the last one

# Remove warnings
import warnings
warnings.filterwarnings("ignore")

# Set the working directory
#import os
#os.chdir('/Users/stellacarneiro/Library/CloudStorage/GoogleDrive-stella.carneiro@loadsmart.com/My Drive/Projects/Data Analysis Club')

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Data manipulation in pandas
import pandas as pd
pd.set_option('display.max_rows', 5000) # Set the maximum number of rows and columns to display
pd.set_option("display.float_format", lambda x: "%.2f" % x) # Number format settings



True

In [ ]:
# Check the api on your navigator:
# https://api.eia.gov/v2/steo/data/?frequency=monthly&data[0]=value&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key=8Uj5YLwHIXwYH0WcIquFhifQRLxfmcZrPredQznw
# you will see they require an api key, so register at https://www.eia.gov/opendata/documentation.php


In [59]:
# Set an api key
# EIA_API_KEY = "[API_CODE_HERE]"

In [60]:
# install requests package
# pip install requests

In [6]:
import requests
import os

# API endpoint URL - the less organized way
# https://www.eia.gov/opendata/browser/steo
# response = requests.get(url) 

url = "https://api.eia.gov/v2/steo/data/?frequency=monthly&data[0]=value&facets[seriesId][]=DSRTUUS&start=2019-01&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000"
url = url + '&api_key=' + os.environ.get("EIA_API_KEY")
url

# Make a GET request
response = requests.get(url)

print(response) # <Response [200]> means it was successful, <Response [400]> means it was not
print(response.status_code) 

# Check the data format, if xml or json
response.headers

# If json
# Download data
data = response.json()

'https://api.eia.gov/v2/steo/data/?frequency=monthly&data[0]=value&facets[seriesId][]=DSRTUUS&start=2019-01&sort[0][column]=period&sort[0][direction]=asc&offset=0&length=5000&api_key=8Uj5YLwHIXwYH0WcIquFhifQRLxfmcZrPredQznw'

<Response [200]>
200


{'Date': 'Fri, 15 Mar 2024 19:18:20 GMT', 'Content-Type': 'application/json', 'Content-Length': '1196', 'Connection': 'keep-alive', 'Access-Control-Allow-Headers': 'Origin, X-Requested-With, Content-Type, Accept, Authorization, X-Params', 'Access-Control-Allow-Methods': 'GET,POST', 'Access-Control-Allow-Origin': '*', 'Age': '0', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Content-Encoding': 'gzip', 'Expires': 'Fri, 15 Mar 2024 19:18:20 GMT', 'Pragma': 'no-cache', 'Strict-Transport-Security': 'max-age=63072000; includeSubdomains;, max-age=31536000; preload', 'Vary': 'Accept-Encoding, Accept-Encoding', 'Via': 'https/1.1 api-umbrella (ApacheTrafficServer [cMsSf ])', 'X-Api-Umbrella-Request-Id': 'ce8e6ghjp9hc1o9esat0', 'X-Cache': 'MISS', 'X-Vcap-Request-Id': '297b717e-2e03-4735-4d78-e772a975e419', 'X-Frame-Options': 'SAMEORIGIN', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block'}

In [7]:
# pandas 
df =  pd.DataFrame(data['response']['data'])
df.head()

,period,seriesId,seriesDescription,value,unit
0,2019-01,DSRTUUS,"Diesel Fuel Retail Price Including Taxes, U.S....",297.975,cents per gallon
1,2019-02,DSRTUUS,"Diesel Fuel Retail Price Including Taxes, U.S....",299.65,cents per gallon
2,2019-03,DSRTUUS,"Diesel Fuel Retail Price Including Taxes, U.S....",307.625,cents per gallon
3,2019-04,DSRTUUS,"Diesel Fuel Retail Price Including Taxes, U.S....",312.1,cents per gallon
4,2019-05,DSRTUUS,"Diesel Fuel Retail Price Including Taxes, U.S....",316.125,cents per gallon


In [10]:
# You can structure a code, if request is sucessful, you can download the data
# Check if the request was successful
if response.status_code == 200: 
    # Parse JSON data
    data = response.json()
    # convert to dataframe
    raw_data = data['response']['data']
    df = pd.DataFrame(raw_data)
else:
    print(f"Failed to fetch data: {response.status_code}")

df.head()

,period,seriesId,seriesDescription,value,unit
0,2019-01,DSRTUUS,"Diesel Fuel Retail Price Including Taxes, U.S....",297.975,cents per gallon
1,2019-02,DSRTUUS,"Diesel Fuel Retail Price Including Taxes, U.S....",299.65,cents per gallon
2,2019-03,DSRTUUS,"Diesel Fuel Retail Price Including Taxes, U.S....",307.625,cents per gallon
3,2019-04,DSRTUUS,"Diesel Fuel Retail Price Including Taxes, U.S....",312.1,cents per gallon
4,2019-05,DSRTUUS,"Diesel Fuel Retail Price Including Taxes, U.S....",316.125,cents per gallon


In [34]:
# convert to excel
df.to_excel('steo_data.xlsx', index=False)

In [46]:
# Another good api is FRED's API because most of US economic data is stored there
# https://fred.stlouisfed.org/docs/api/fred/

# Set the base URL for the FED API
base_url = 'https://api.stlouisfed.org/fred/series/observations'

# Set the variables to be requested
#vars = pd.DataFrame({'variable':['Real Personal Consumption Expenditures'],
#                     'code': ['DPCERL1Q225SBEA']})

code = 'GASDESW'
desc_code = 'US Diesel Sales Price, Dol/Gal, NSA, Weekly'

import requests

# Set the parameters for the API request
params = {
    'api_key': os.environ.get("FRED_API_KEY"),
    'series_id': code,
    'observation_start': '2020-01-01',
   # 'observation_end':'2023-01-01'
}

# Make the API request

response = requests.get(base_url, params=params)
response

print(response.url)

<Response [200]>

https://api.stlouisfed.org/fred/series/observations?api_key=59af226442f05b4b5c606095a06074f0&series_id=GASDESW&observation_start=2020-01-01


In [42]:
# Check the content of the response
response.text
response.content

# See data format
response.headers

# As it is xml, convert with pandas function
df = pd.read_xml(response.text)
df.head()

'<?xml version="1.0" encoding="utf-8" ?>\n<observations realtime_start="2024-03-15" realtime_end="2024-03-15" observation_start="2020-01-01" observation_end="9999-12-31" units="lin" output_type="1" file_type="xml" order_by="observation_date" sort_order="asc" count="219" offset="0" limit="100000">\n  <observation realtime_start="2024-03-15" realtime_end="2024-03-15" date="2020-01-06" value="3.079"/>\n  <observation realtime_start="2024-03-15" realtime_end="2024-03-15" date="2020-01-13" value="3.064"/>\n  <observation realtime_start="2024-03-15" realtime_end="2024-03-15" date="2020-01-20" value="3.037"/>\n  <observation realtime_start="2024-03-15" realtime_end="2024-03-15" date="2020-01-27" value="3.01"/>\n  <observation realtime_start="2024-03-15" realtime_end="2024-03-15" date="2020-02-03" value="2.956"/>\n  <observation realtime_start="2024-03-15" realtime_end="2024-03-15" date="2020-02-10" value="2.91"/>\n  <observation realtime_start="2024-03-15" realtime_end="2024-03-15" date="2020

In [48]:
df['variable'] = desc_code
df.head()

,realtime_start,realtime_end,date,value,variable
0,2024-03-15,2024-03-15,2020-01-06,3.08,"US Diesel Sales Price, Dol/Gal, NSA, Weekly"
1,2024-03-15,2024-03-15,2020-01-13,3.06,"US Diesel Sales Price, Dol/Gal, NSA, Weekly"
2,2024-03-15,2024-03-15,2020-01-20,3.04,"US Diesel Sales Price, Dol/Gal, NSA, Weekly"
3,2024-03-15,2024-03-15,2020-01-27,3.01,"US Diesel Sales Price, Dol/Gal, NSA, Weekly"
4,2024-03-15,2024-03-15,2020-02-03,2.96,"US Diesel Sales Price, Dol/Gal, NSA, Weekly"


In [49]:
df.to_excel('fred.xlsx')